<a href="https://colab.research.google.com/github/sazio/MultiAgentCovid/blob/master/People_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np 
import pandas as pd # data handling
import glob # folder and path handling
from tqdm.notebook import tqdm # measuring for loops runtime
from scipy import sparse # to export sparse adjacency matrices
import matplotlib.pyplot as plt # plot 

import warnings
warnings.filterwarnings('ignore')

In [32]:
# Import data from GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [33]:
path = "drive/MyDrive/MAS/Data/Output_GAMA/Data" # path to folder & files
people_path = path + "/People" # path to agents files , gama export
num_people = 1 #30030 
betas = [0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15]
#betas = [0.025, 0.05, 0.075, 0.1]

people_files = []

for i in tqdm(range(0,num_people)):
  people_files.append(people_path + "/people" + str(i) + ".txt")

In [26]:
def preprocess_people(people_file):

  DF_people = pd.read_csv(people_files[0], sep = ",", names = ["cycle","beta", "working_place","living_place","is_infected","is_immune","is_dead"])
  DF_people = DF_people[DF_people["cycle"] != "cycle"]
  DF_people = DF_people.reset_index(drop = True)

  DF_people["cycle"] = DF_people["cycle"].astype(np.int16)  # dtype conversion
  DF_people["beta"] = DF_people["beta"].astype(np.float32)
  DF_people = DF_people.replace({"is_infected":{"true":1, "false":0}})
  DF_people = DF_people.replace({"is_immune":{"true":1, "false":0}})
  DF_people = DF_people.replace({"is_dead":{"true":1, "false":0}}) 

  DF_people["is_susceptible"] = 1 - (DF_people.is_infected + DF_people.is_immune + DF_people.is_dead)

  mean_ =  DF_people.groupby(['beta', 'cycle'])['is_susceptible', 'is_immune', 'is_infected', 'is_dead'].mean()
  var_  =  DF_people.groupby(['beta', 'cycle'])['is_susceptible', 'is_immune', 'is_infected', 'is_dead'].var()

return mean_, var_